<a href="https://colab.research.google.com/github/kweenkeen/ReachOutandRead/blob/main/ROR_IHS_Biannual_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter
!pip install openpyxl

import pandas as pd
from google.colab import files
from datetime import date
from datetime import datetime

# Make sure to upload site data report for TWO REPORTING PERIODS!

site_data_report = files.upload()
site_data_file = next(iter(site_data_report))
df = pd.read_excel(site_data_file, converters={'Site ID': str})

     |████████████████████████████████| 149 kB 11.3 MB/s 


Saving MyROR Site Data Report 2022-03-08.xlsx to MyROR Site Data Report 2022-03-08.xlsx


In [2]:
df = df.set_index('Site ID')

In [3]:
df = df[['Site Type', 'Site Name', 'Status','Join Date', 'Inactive Date', 'IHS', 'Categories', 'Physical Location: City', 'Physical Location: State', 'Report 1:  Name', 'Report 2:  Name', 'Combined Annual Well-child Visits 0-5 Years', 'Combined Annual Books 0-5 Years']]
df = df.rename({'Physical Location: City': 'City', 'Physical Location: State': 'State' , 'Combined Annual Well-child Visits 0-5 Years': 'Visits 0-5 years Annually', 'Combined Annual Books 0-5 Years': 'Books 0-5 Years Annually'}, axis=1)
df['Book Distribution Rate'] = (df['Books 0-5 Years Annually'] / df['Visits 0-5 years Annually']) * 100
df['Book Distribution Rate'] = df['Book Distribution Rate'].fillna(0)
df['Book Distribution Rate'] = df['Book Distribution Rate'].astype(float).round(2)

In [4]:
AIAN_list = ['Grant Funded Sites, ROR American Indian/Alaska Native', 'ROR American Indian/Alaska Native', 'ROR American Indian/Alaska Native, Grant Funded Sites', 'ROR American Indian/Alaska Native, Navajo Nation', 'ROR American Indian/Alaska Native, SI: DD']
df = df[(df["IHS"]=="Y") | (df["Categories"].isin(AIAN_list))]

In [5]:
df.head()

,Site Type,Site Name,Status,Join Date,Inactive Date,IHS,Categories,City,State,Report 1: Name,Report 2: Name,Visits 0-5 years Annually,Books 0-5 Years Annually,Book Distribution Rate
Site ID,,,,,,,,,,,,,,
24,Primary,Alaska Native Medical Center / PCC-Pediatric C...,Active - Non-Reporting,1997-04-01,NaT,N,"Grant Funded Sites, ROR American Indian/Alaska...",Anchorage,AK,January 2020 Progress Report,July 2020 Progress Report,3610,2956,81.88
31,Primary,Chinle Comprehensive Health Care Facility /,Active,1999-07-01,NaT,Y,"ROR American Indian/Alaska Native, Navajo Nation",Chinle,AZ,January 2022 Progress Report,July 2019 Progress Report,2252,1752,77.80
33,Primary,Sage Memorial Hospital / Navajo Reach Out and ...,Inactivated,1996-12-01,2018-11-07,Y,"ROR American Indian/Alaska Native, Navajo Nation",Ganado,AZ,January 2009 Progress Report,July 2008 Progress Report,198,207,104.55
36,Primary,Kayenta Health Center /,Active,2000-10-01,NaT,Y,"ROR American Indian/Alaska Native, Navajo Nation",Kayenta,AZ,January 2022 Progress Report,July 2021 Progress Report,385,385,100.00
36.3,Satellite,Inscription House,Inactivated,2011-08-01,NaT,NaN,"ROR American Indian/Alaska Native, Navajo Nation",Shonto,AZ,NaN,NaN,0,0,0.00


In [6]:
high_level_active = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze'] 
IHS_AIAN_df = df[df['Status'].isin(high_level_active)]
IHS_AIAN_df = IHS_AIAN_df[['Site Type', 'Site Name', 'Status', 'City', 'State', 'Report 1:  Name', 'Report 2:  Name', 'Visits 0-5 years Annually', 'Books 0-5 Years Annually', 'Book Distribution Rate']]


In [7]:
today = str(date.today())
filename = 'IHS_AIAN_Data_' + today + '.xlsx'
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

In [8]:
IHS_AIAN_df.head()

,Site Type,Site Name,Status,City,State,Report 1: Name,Report 2: Name,Visits 0-5 years Annually,Books 0-5 Years Annually,Book Distribution Rate
Site ID,,,,,,,,,,
24,Primary,Alaska Native Medical Center / PCC-Pediatric C...,Active - Non-Reporting,Anchorage,AK,January 2020 Progress Report,July 2020 Progress Report,3610,2956,81.88
31,Primary,Chinle Comprehensive Health Care Facility /,Active,Chinle,AZ,January 2022 Progress Report,July 2019 Progress Report,2252,1752,77.80
36,Primary,Kayenta Health Center /,Active,Kayenta,AZ,January 2022 Progress Report,July 2021 Progress Report,385,385,100.00
51,Primary,Tuba City Regional Health Care Corporation,Active,Tuba City,AZ,January 2022 Progress Report,July 2021 Progress Report,2718,1651,60.74
56,Primary,Tohono O'odham Nation Health Center /,Active,Sells,AZ,January 2022 Progress Report,July 2021 Progress Report,2000,2000,100.00


In [9]:
Active_df = IHS_AIAN_df.loc[IHS_AIAN_df['Status'] == 'Active']
Active_df['Book Distribution Rate'] = Active_df['Book Distribution Rate'].astype(str) + '%'
Active_df.to_excel(writer, sheet_name="Active")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
join_date_range_min = input("Type in the start of your year (MM/DD/YYYY): ")
join_date_range_min = datetime.strptime(join_date_range_min, '%m/%d/%Y')
#join_date_range_min = join_date_range_min.strftime('%m/%d/%Y')
join_date_range_max = input("Type in the end of your year (MM/DD/YYYY): ")
join_date_range_max = datetime.strptime(join_date_range_max, '%m/%d/%Y')
#join_date_range_max = join_date_range_max.strftime('%m/%d/%Y')
print(type(join_date_range_min))

Type in the start of your year (MM/DD/YYYY): 01/01/2021
Type in the end of your year (MM/DD/YYYY): 12/31/2021
<class 'datetime.datetime'>


In [11]:
df.head()

,Site Type,Site Name,Status,Join Date,Inactive Date,IHS,Categories,City,State,Report 1: Name,Report 2: Name,Visits 0-5 years Annually,Books 0-5 Years Annually,Book Distribution Rate
Site ID,,,,,,,,,,,,,,
24,Primary,Alaska Native Medical Center / PCC-Pediatric C...,Active - Non-Reporting,1997-04-01,NaT,N,"Grant Funded Sites, ROR American Indian/Alaska...",Anchorage,AK,January 2020 Progress Report,July 2020 Progress Report,3610,2956,81.88
31,Primary,Chinle Comprehensive Health Care Facility /,Active,1999-07-01,NaT,Y,"ROR American Indian/Alaska Native, Navajo Nation",Chinle,AZ,January 2022 Progress Report,July 2019 Progress Report,2252,1752,77.80
33,Primary,Sage Memorial Hospital / Navajo Reach Out and ...,Inactivated,1996-12-01,2018-11-07,Y,"ROR American Indian/Alaska Native, Navajo Nation",Ganado,AZ,January 2009 Progress Report,July 2008 Progress Report,198,207,104.55
36,Primary,Kayenta Health Center /,Active,2000-10-01,NaT,Y,"ROR American Indian/Alaska Native, Navajo Nation",Kayenta,AZ,January 2022 Progress Report,July 2021 Progress Report,385,385,100.00
36.3,Satellite,Inscription House,Inactivated,2011-08-01,NaT,NaN,"ROR American Indian/Alaska Native, Navajo Nation",Shonto,AZ,NaN,NaN,0,0,0.00


In [12]:
df['Join Date'] = pd.to_datetime(df['Join Date'], format='%m/%d/%Y')

In [13]:
df.head()

,Site Type,Site Name,Status,Join Date,Inactive Date,IHS,Categories,City,State,Report 1: Name,Report 2: Name,Visits 0-5 years Annually,Books 0-5 Years Annually,Book Distribution Rate
Site ID,,,,,,,,,,,,,,
24,Primary,Alaska Native Medical Center / PCC-Pediatric C...,Active - Non-Reporting,1997-04-01,NaT,N,"Grant Funded Sites, ROR American Indian/Alaska...",Anchorage,AK,January 2020 Progress Report,July 2020 Progress Report,3610,2956,81.88
31,Primary,Chinle Comprehensive Health Care Facility /,Active,1999-07-01,NaT,Y,"ROR American Indian/Alaska Native, Navajo Nation",Chinle,AZ,January 2022 Progress Report,July 2019 Progress Report,2252,1752,77.80
33,Primary,Sage Memorial Hospital / Navajo Reach Out and ...,Inactivated,1996-12-01,2018-11-07,Y,"ROR American Indian/Alaska Native, Navajo Nation",Ganado,AZ,January 2009 Progress Report,July 2008 Progress Report,198,207,104.55
36,Primary,Kayenta Health Center /,Active,2000-10-01,NaT,Y,"ROR American Indian/Alaska Native, Navajo Nation",Kayenta,AZ,January 2022 Progress Report,July 2021 Progress Report,385,385,100.00
36.3,Satellite,Inscription House,Inactivated,2011-08-01,NaT,NaN,"ROR American Indian/Alaska Native, Navajo Nation",Shonto,AZ,NaN,NaN,0,0,0.00


In [14]:
print(type(join_date_range_min))
print(join_date_range_max)

<class 'datetime.datetime'>
2021-12-31 00:00:00


In [15]:
join_date_df = df.loc[(df['Join Date'] >= join_date_range_min) & (df['Join Date'] <= join_date_range_max)]
join_date_df['Join Date'] = join_date_df['Join Date'].dt.date
join_date_df['Book Distribution Rate'] = join_date_df['Book Distribution Rate'].astype(str) + '%'
join_date_df = join_date_df[['Site Type', 'Site Name', 'Status', 'City', 'State', 'Report 1:  Name', 'Report 2:  Name', 'Visits 0-5 years Annually', 'Books 0-5 Years Annually', 'Book Distribution Rate', 'Join Date']]
join_date_df.to_excel(writer, sheet_name="New Sites")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
join_date_df.head()

,Site Type,Site Name,Status,City,State,Report 1: Name,Report 2: Name,Visits 0-5 years Annually,Books 0-5 Years Annually,Book Distribution Rate,Join Date
Site ID,,,,,,,,,,,
2147.4,Satellite,Snowbird Clinic / CIHA PEDS,Active,Robbinsville,NC,January 2022 Progress Report,January 2022 Progress Report,10,10,100.0%,2021-09-13
6031.3,Satellite,Native Health Mesa /,Active,Mesa,AZ,January 2022 Progress Report,January 2022 Progress Report,80,76,95.0%,2021-12-06
8676.3,Satellite,Lacreek District IHS Clinic / Attention: Charl...,Active,"Martin, SD",SD,January 2022 Progress Report,January 2022 Progress Report,156,56,35.9%,2021-09-13
18810,Primary,Eastern Aleutian Tribes /,Active,Anchorage,AK,Site Application,Site Application,288,288,100.0%,2021-07-24


In [17]:
Book_Distribution_Gaps_df = IHS_AIAN_df.loc[IHS_AIAN_df['Book Distribution Rate'] < 80]
Book_Distribution_Gaps_df['Book Distribution Rate'] = Book_Distribution_Gaps_df['Book Distribution Rate'].astype(str) + '%'
Book_Distribution_Gaps_df.to_excel(writer, sheet_name="Sites w. Book Distribution Gaps")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
IHS_AIAN_df['Book Distribution Rate'] = IHS_AIAN_df['Book Distribution Rate'].astype(str) + '%'

In [19]:
prospective_sites = ['Approved - Needs Training', 'Approved - Waitlisted'] 
Prospective_Sites_df = df[df['Status'].isin(prospective_sites)] 
Prospective_Sites_df = Prospective_Sites_df[['Site Type', 'Site Name', 'Status', 'City', 'State', 'Report 1:  Name', 'Report 2:  Name', 'Visits 0-5 years Annually', 'Books 0-5 Years Annually', 'Book Distribution Rate']]
Prospective_Sites_df.to_excel(writer, sheet_name="Prospective Sites")

In [20]:
Prospective_Sites_df.head()

,Site Type,Site Name,Status,City,State,Report 1: Name,Report 2: Name,Visits 0-5 years Annually,Books 0-5 Years Annually,Book Distribution Rate
Site ID,,,,,,,,,,
13005,Primary,Dilkon Clinic / WIHCC - Dilkon Clinic,Approved - Needs Training,Winslow,AZ,Site Application,Site Application,90,90,100.0
16835,Primary,Leupp Clinic /,Approved - Needs Training,Leupp,AZ,Site Application,Site Application,90,90,100.0
18065,Primary,Sacred Circle Healthcare /,Approved - Waitlisted,Salt Lake City,UT,Site Application,Site Application,400,400,100.0


In [21]:
Non_Reporting_df = IHS_AIAN_df.loc[IHS_AIAN_df['Status'] == 'Active - Non-Reporting']
Non_Reporting_df.to_excel(writer, sheet_name="Non-Reporting Sites")

In [22]:
COVID_Freeze_df = IHS_AIAN_df.loc[IHS_AIAN_df['Status'] == 'COVID-19 Freeze']
COVID_Freeze_df.to_excel(writer, sheet_name="COVID-19 Freeze")

In [23]:
S4D_df = IHS_AIAN_df.loc[IHS_AIAN_df['Status'] == 'Slated for deactivation']
S4D_df.to_excel(writer, sheet_name="In Danger of Deactivation")

In [24]:
deactivated_date_df = df.loc[(df['Inactive Date'] >= join_date_range_min) & (df['Inactive Date'] <= join_date_range_max)]
deactivated_date_df['Inactive Date'] = deactivated_date_df['Inactive Date'].dt.date
deactivated_date_df['Book Distribution Rate'] = deactivated_date_df['Book Distribution Rate'].astype(str) + '%'
deactivated_date_df = deactivated_date_df[['Site Type', 'Site Name', 'Status', 'City', 'State', 'Report 1:  Name', 'Report 2:  Name', 'Visits 0-5 years Annually', 'Books 0-5 Years Annually', 'Book Distribution Rate', 'Inactive Date']]
deactivated_date_df.to_excel(writer, sheet_name="Recently Deactivated")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
# IHS_AIAN_df.to_excel(writer, sheet_name="IHS and AIAN Sites")
writer.save()
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>